In [ ]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple -U modelscope addict

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com


In [18]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")

from pprint import pp
from modelscope.msdatasets import MsDataset
from datasets import (load_dataset, load_from_disk, Dataset)

In [8]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_model = "F:/LLM"
path_output = os.path.join(path_model, "output")

In [ ]:
# dataset =  MsDataset.load(os.path.join(path_data, "ShareGPT/ShareGPT_V3_unfiltered_cleaned_split.json"), subset_name="default", split="train")
dataset = load_dataset(
    path="json",
    data_files=os.path.join(path_data, "ShareGPT/ShareGPT_V3_unfiltered_cleaned_split.json"),
    split="all"
)

2025-04-26 20:22:25,681 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from C:/my_project/MyGit/Machine-Learning-Column\data\ShareGPT/ShareGPT_V3_unfiltered_cleaned_split.json. Please make sure that you can trust the external codes.
2025-04-26 20:22:25,682 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from json. Please make sure that you can trust the external codes.


Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
dataset = load_dataset(
    path="json",
    data_files=os.path.join(path_data, "ShareGPT/ShareGPT_V3_unfiltered_cleaned_split.json"),
    split="all"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
dataset[1]

{'id': 'i6IyJda_0',
 'conversations': [{'from': 'human',
   'markdown': None,
   'text': None,
   'value': 'How to tell if a customer segment is well segmented? In 3 bullet points.'},
  {'from': 'gpt',
   'markdown': None,
   'text': None,
   'value': '1. Homogeneity: The segment should consist of customers who share similar characteristics and behaviors.\n2. Distinctiveness: The segment should be different from other segments in terms of their characteristics and behaviors.\n3. Stability: The segment should remain relatively stable over time and not change drastically. The characteristics and behaviors of customers within the segment should not change significantly.'}]}